In [1]:
import numpy as np
from keras_resnet.models import ResNet18
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.applications.resnet50 import preprocess_input
from sklearn.preprocessing import normalize
from sklearn.metrics.pairwise import cosine_similarity
from tensorflow.image import resize
import tensorflow as tf
import copy
import pickle
import time

In [2]:
# Initialize the MobileNetV2 model for feature extraction
# Input shape is set to (96, 96, 3) for a balance between speed and feature quality
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
feature_extractor = MobileNetV2(weights='imagenet', include_top=False, pooling='avg', input_shape=(96, 96, 3))

def extract_features(data):
    """
    Extracts features from image data using the pre-trained MobileNetV2 model.

    Args:
        data (numpy.ndarray): Array of image data with shape (num_samples, 32, 32, 3).

    Returns:
        numpy.ndarray: Extracted feature vectors.
    """
    # Resize images from (32, 32) to (96, 96) as MobileNetV2 expects larger images
    resized_data = tf.image.resize(data, (96, 96)).numpy()
    preprocessed_data = preprocess_input(resized_data)

    # Extract features in batches to optimize memory usage
    batch_size = 64
    num_samples = preprocessed_data.shape[0]
    features = []

    for i in range(0, num_samples, batch_size):
        batch = preprocessed_data[i:i+batch_size]
        batch_features = feature_extractor.predict(batch, verbose=0)
        # Normalize features for cosine similarity
        batch_features = normalize(batch_features)
        features.append(batch_features)

    features = np.vstack(features)
    return features


In [20]:
class LwPClassifier:
    def __init__(self, num_classes, feature_dim):
        """
        Initializes the LwP classifier with empty prototypes.

        Args:
            num_classes (int): Number of classes.
            feature_dim (int): Dimensionality of feature vectors.
        """
        self.num_classes = num_classes
        self.feature_dim = feature_dim
        # Initialize prototypes as zero vectors
        self.prototypes = np.zeros((num_classes, feature_dim))
        # Count of samples per class for prototype updating
        self.class_counts = np.zeros(num_classes)
    
    def predict_probabilities(self, features):
        """
        Computes normalized cosine similarity scores as probabilities.

        Args:
            features (numpy.ndarray): Feature vectors.

        Returns:
            numpy.ndarray: Probability-like scores for each class.
        """
        similarities = cosine_similarity(features, self.prototypes)
        # Normalize similarities to range [0, 1] to act as probabilities
        exp_similarities = np.exp(similarities)  # Exponentiate to ensure positivity
        probabilities = exp_similarities / np.sum(exp_similarities, axis=1, keepdims=True)
        return probabilities

    def initialize_prototypes(self, features, labels):
        """
        Initializes class prototypes based on the initial labeled data.

        Args:
            features (numpy.ndarray): Feature vectors.
            labels (numpy.ndarray): Corresponding labels.
        """
        for class_idx in range(self.num_classes):
            class_features = features[labels == class_idx]
            if len(class_features) > 0:
                self.prototypes[class_idx] = np.mean(class_features, axis=0)
                self.prototypes[class_idx] /= np.linalg.norm(self.prototypes[class_idx])  # Normalize
                self.class_counts[class_idx] = len(class_features)
            else:
                # Handle classes with no initial samples
                self.prototypes[class_idx] = np.random.randn(self.feature_dim)
                self.prototypes[class_idx] /= np.linalg.norm(self.prototypes[class_idx])
                self.class_counts[class_idx] = 1

    def predict(self, features):
        """
        Predicts class labels for given feature vectors.

        Args:
            features (numpy.ndarray): Feature vectors.

        Returns:
            numpy.ndarray: Predicted class labels.
        """
        similarities = cosine_similarity(features, self.prototypes)
        return np.argmax(similarities, axis=1)

    def update_prototypes(self, features, labels):
        """
        Updates class prototypes with new labeled data.

        Args:
            features (numpy.ndarray): Newly labeled feature vectors.
            labels (numpy.ndarray): Corresponding labels.
        """
        for class_idx in range(self.num_classes):
            class_features = features[labels == class_idx]
            if len(class_features) == 0:
                continue  # No new data for this class
            # Compute mean of new features
            new_mean = np.mean(class_features, axis=0)
            new_mean /= np.linalg.norm(new_mean)
            # Update prototype using running average
            total_count = self.class_counts[class_idx] + len(class_features)
            self.prototypes[class_idx] = (self.prototypes[class_idx] * self.class_counts[class_idx] + new_mean * len(class_features)) / total_count
            self.prototypes[class_idx] /= np.linalg.norm(self.prototypes[class_idx])  # Normalize
            self.class_counts[class_idx] = total_count


In [4]:
import torch
D1 = torch.load('1_train_data.tar.pth')
#print(t.keys()) # it will print data and targets
#data, targets = t['data'], t['targets'] # both numpy.ndarray
D2 = torch.load('2_train_data.tar.pth')
D3 = torch.load('3_train_data.tar.pth')
D4 = torch.load('4_train_data.tar.pth')
D5 = torch.load('5_train_data.tar.pth')
D6 = torch.load('6_train_data.tar.pth')
D7 = torch.load('7_train_data.tar.pth')
D8 = torch.load('8_train_data.tar.pth')
D9 = torch.load('9_train_data.tar.pth')
D10 = torch.load('10_train_data.tar.pth')
D11 = torch.load('11_train_data.tar.pth')
D12 = torch.load('12_train_data.tar.pth')
D13 = torch.load('13_train_data.tar.pth')
D14 = torch.load('14_train_data.tar.pth')
D15 = torch.load('15_train_data.tar.pth')
D16 = torch.load('16_train_data.tar.pth')
D17 = torch.load('17_train_data.tar.pth')
D18 = torch.load('18_train_data.tar.pth')
D19 = torch.load('19_train_data.tar.pth')
D20 = torch.load('20_train_data.tar.pth')

C:\Users\AVNI MAHESHWARI\AppData\Local\Temp\ipykernel_18120\2379473238.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  D1 = torch.load('1_train_data.tar.pth')
C:\Users\A

C:\Users\AVNI MAHESHWARI\AppData\Local\Temp\ipykernel_18120\2379473238.py:13: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  D10 = torch.load('10_train_data.tar.pth')
C:\User

In [5]:
T1 = torch.load('1_eval_data.tar.pth')
T2 = torch.load('2_eval_data.tar.pth')
T3 = torch.load('3_eval_data.tar.pth')
T4 = torch.load('4_eval_data.tar.pth')
T5 = torch.load('5_eval_data.tar.pth')
T6 = torch.load('6_eval_data.tar.pth')
T7 = torch.load('7_eval_data.tar.pth')
T8 = torch.load('8_eval_data.tar.pth')
T9 = torch.load('9_eval_data.tar.pth')
T10 = torch.load('10_eval_data.tar.pth')
T11 = torch.load('11_eval_data.tar.pth')
T12 = torch.load('12_eval_data.tar.pth')
T13 = torch.load('13_eval_data.tar.pth')
T14 = torch.load('14_eval_data.tar.pth')
T15 = torch.load('15_eval_data.tar.pth')
T16 = torch.load('16_eval_data.tar.pth')
T17 = torch.load('17_eval_data.tar.pth')
T18 = torch.load('18_eval_data.tar.pth')
T19 = torch.load('19_eval_data.tar.pth')
T20 = torch.load('20_eval_data.tar.pth')

C:\Users\AVNI MAHESHWARI\AppData\Local\Temp\ipykernel_18120\3206332515.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  T1 = torch.load('1_eval_data.tar.pth')
C:\Users\AV

C:\Users\AVNI MAHESHWARI\AppData\Local\Temp\ipykernel_18120\3206332515.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  T11 = torch.load('11_eval_data.tar.pth')
C:\Users

C:\Users\AVNI MAHESHWARI\AppData\Local\Temp\ipykernel_18120\3206332515.py:20: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  T20 = torch.load('20_eval_data.tar.pth')


In [6]:
Testsets = []
data = T1['data']  # Shape: (num_samples, 32, 32, 3)
target = T1['targets']  # Shape: (num_samples,)
Testsets.append({'data': data, 'target': target})
data = T2['data']
target = T2['targets']
Testsets.append({'data': data, 'target': target})
data = T3['data']
target = T3['targets']
Testsets.append({'data': data, 'target': target})
data = T4['data']
target = T4['targets']
Testsets.append({'data': data, 'target': target})
data = T5['data']
target = T5['targets']
Testsets.append({'data': data, 'target': target})
data = T6['data']
target = T6['targets']
Testsets.append({'data': data, 'target': target})
data = T7['data']
target = T7['targets']
Testsets.append({'data': data, 'target': target})
data = T8['data']
target = T8['targets']
Testsets.append({'data': data, 'target': target})
data = T9['data']
target = T9['targets']
Testsets.append({'data': data, 'target': target})
data = T10['data']
target = T10['targets']
Testsets.append({'data': data, 'target': target})
data = T11['data']
target = T11['targets']
Testsets.append({'data': data, 'target': target})
data = T12['data']
target = T12['targets']
Testsets.append({'data': data, 'target': target})
data = T13['data']
target = T13['targets']
Testsets.append({'data': data, 'target': target})
data = T14['data']
target = T14['targets']
Testsets.append({'data': data, 'target': target})
data = T15['data']
target = T15['targets']
Testsets.append({'data': data, 'target': target})
data = T16['data']
target = T16['targets']
Testsets.append({'data': data, 'target': target})
data = T17['data']
target = T17['targets']
Testsets.append({'data': data, 'target': target})
data = T18['data']
target = T18['targets']
Testsets.append({'data': data, 'target': target})
data = T19['data']
target = T19['targets']
Testsets.append({'data': data, 'target': target})
data = T20['data']
target = T20['targets']
Testsets.append({'data': data, 'target': target})

In [7]:
datasets = []
data = D1['data']  # Shape: (num_samples, 32, 32, 3)
target = D1['targets']  # Shape: (num_samples,)
datasets.append({'data': data, 'target': target})

data = D2['data']  # Shape: (num_samples, 32, 32,)
datasets.append({'data': data})
data = D3['data']
datasets.append({'data': data})
data = D4['data']
datasets.append({'data': data})
data = D5['data']
datasets.append({'data': data})
data = D6['data']
datasets.append({'data': data})
data = D7['data']
datasets.append({'data': data})
data = D8['data']
datasets.append({'data': data})
data = D9['data']
datasets.append({'data': data})
data = D10['data']
datasets.append({'data': data})


In [8]:
for i in range(10):  # Only first 10 datasets as per the problem
    #print(f"Extracting features for D{i+1}...")
    data = datasets[i]['data']
    #feature_extractor = initialize_custom_cnn()
    features = extract_features(data)
    datasets[i]['features'] = features
    if i == 0:
        labels = datasets[i]['target']
        datasets[i]['labels'] = labels

In [9]:
for i in range(20):  # Only first 10 datasets as per the problem
    #print(f"Extracting features for D{i+1}...")
    data = Testsets[i]['data']
    #feature_extractor = initialize_feature_extractor()
    #features = extract_features(data,batch_size=128)
    #feature_extractor = initialize_custom_cnn()
    features = extract_features(data)
    #features = extract_features(data)
    Testsets[i]['features'] = features
    labels = Testsets[i]['target']
    Testsets[i]['labels'] = labels

In [27]:
num_classes = 10  # CIFAR-10 has 10 classes
feature_dim = 1280  # ResNet50 with pooling='avg' outputs 2048-dimensional vectors

# Initialize LwP classifier
lwpc = LwPClassifier(num_classes=num_classes, feature_dim=feature_dim)
#lwpc = SigmoidKernelLwPClassifier(num_classes=10, feature_dim=1280, alpha=1.0, c=0.05)

# Initialize prototypes using D1
lwpc.initialize_prototypes(datasets[0]['features'], datasets[0]['labels'])

models = []
models.append(lwpc)

In [32]:
# Initialize prototypes using D1
#lwpc = LwPClassifier(num_classes=num_classes, feature_dim=feature_dim)
#lwpc.initialize_prototypes(datasets[0]['features'], datasets[0]['labels'])

for i in range(0, 10):  # Indices 1 to 9 correspond to D2 to D10
    print("#########################")
    print(f"testing for {i+1}th model")
    print("#########################")
    
    if i!=0 :
        current_features = datasets[i]['features']
        predicted_labels = lwpc.predict(current_features)
        datasets[i]['predicted_labels'] = predicted_labels
        lwpc.update_prototypes(current_features, predicted_labels)
        models.append(lwpc)
        
    for j in range(0,i+1):
           feature = Testsets[j]['features']
           label = Testsets[j]['labels']

           predictions = lwpc.predict(feature)
           accuracy = np.mean(predictions == label)
           print(f"Accuracy on T{j+1}: {accuracy}")

#########################
testing for 1th model
#########################
Accuracy on T1: 0.7504
#########################
testing for 2th model
#########################
Accuracy on T1: 0.7504
Accuracy on T2: 0.7488
#########################
testing for 3th model
#########################
Accuracy on T1: 0.7476
Accuracy on T2: 0.7484
Accuracy on T3: 0.7304
#########################
testing for 4th model
#########################
Accuracy on T1: 0.7484
Accuracy on T2: 0.748
Accuracy on T3: 0.7304
Accuracy on T4: 0.7468
#########################
testing for 5th model
#########################
Accuracy on T1: 0.7488
Accuracy on T2: 0.7476
Accuracy on T3: 0.7292
Accuracy on T4: 0.7468
Accuracy on T5: 0.764
#########################
testing for 6th model
#########################
Accuracy on T1: 0.7472
Accuracy on T2: 0.748
Accuracy on T3: 0.7308
Accuracy on T4: 0.7452
Accuracy on T5: 0.7624
Accuracy on T6: 0.7612
#########################
testing for 7th model
#########################
Ac

In [12]:
D11 = torch.load('11_train_data.tar.pth')
D12 = torch.load('12_train_data.tar.pth')
D13 = torch.load('13_train_data.tar.pth')
D14 = torch.load('14_train_data.tar.pth')
D15 = torch.load('15_train_data.tar.pth')
D16 = torch.load('16_train_data.tar.pth')
D17 = torch.load('17_train_data.tar.pth')
D18 = torch.load('18_train_data.tar.pth')
D19 = torch.load('19_train_data.tar.pth')
D20 = torch.load('20_train_data.tar.pth')

C:\Users\AVNI MAHESHWARI\AppData\Local\Temp\ipykernel_18120\2243097193.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  D11 = torch.load('11_train_data.tar.pth')
C:\Users

In [13]:
data = D11['data']  # Shape: (num_samples, 32, 32,)
datasets.append({'data': data})
data = D12['data']
datasets.append({'data': data})
data = D13['data']
datasets.append({'data': data})
data = D14['data']
datasets.append({'data': data})
data = D15['data']
datasets.append({'data': data})
data = D16['data']
datasets.append({'data': data})
data = D17['data']
datasets.append({'data': data})
data = D18['data']
datasets.append({'data': data})
data = D19['data']
datasets.append({'data': data})
data = D20['data']
datasets.append({'data': data})

In [14]:
for i in range(10,20):  # Only first 10 datasets as per the problem
    #print(f"Extracting features for D{i+1}...")
    data = datasets[i]['data']
    features = extract_features(data)
    datasets[i]['features'] = features
    if i == 0:
        labels = datasets[i]['target']
        datasets[i]['labels'] = labels

In [31]:
len(models)

10

In [33]:

import torch
import numpy as np

# Updated pseudo-labeling mechanism
def pseudo_label_with_confidence(model, features, confidence_threshold=0.8):
    """
    Generates pseudo-labels for features using the LwPClassifier.
    Only returns samples with a confidence above the threshold.

    Args:
        model (LwPClassifier): The classifier.
        features (numpy.ndarray): Feature vectors.
        confidence_threshold (float): Minimum confidence for pseudo-labeling.

    Returns:
        numpy.ndarray, numpy.ndarray: Confident features and their pseudo-labels.
    """
    # Get probability-like scores
    probabilities = model.predict_probabilities(features)

    # Compute confidences and pseudo-labels
    confidences = np.max(probabilities, axis=1)
    pseudo_labels = np.argmax(probabilities, axis=1)

    # Filter samples with confidence above the threshold
    mask = confidences >= confidence_threshold
    confident_features = features[mask]
    confident_labels = pseudo_labels[mask]
    return confident_features, confident_labels

# Mixup augmentation function
def mixup(x1, x2, y1, y2, alpha=0.4):
    lam = np.random.beta(alpha, alpha)
    x_mix = lam * x1 + (1 - lam) * x2
    y_mix = lam * y1 + (1 - lam) * y2
    return x_mix, y_mix

# Contrastive alignment loss
def prototype_alignment_loss(features, prototypes, labels):
    similarities = torch.matmul(features, prototypes.T)
    loss = torch.nn.CrossEntropyLoss()(similarities, labels)
    return loss

# Update process with improved mechanisms
for i in range(10, 20):  # Task 1.2 updates for D11 to D20
    print(f"Updating model for Dataset D{i+1}...")

    # Extract features and pseudo-labels with confidence filtering
    current_features = datasets[i]['features']
    confident_features, pseudo_labels = pseudo_label_with_confidence(lwpc, current_features)

    # Apply Random Mixup
    if confident_features.shape[0] > 1:  # Ensure at least two samples for mixup
        idx = np.random.permutation(confident_features.shape[0])  # Use numpy for shuffling indices
        mixed_features, mixed_labels = mixup(
            confident_features, confident_features[idx],
            pseudo_labels, pseudo_labels[idx]
        )
    else:
        mixed_features, mixed_labels = confident_features, pseudo_labels

    # Update prototypes using contrastive alignment
    lwpc.update_prototypes(mixed_features, mixed_labels)
    models.append(lwpc)

    # Evaluate the updated model
    for j in range(0, i + 1):  # Test on all seen datasets
        features = Testsets[j]['features']
        labels = Testsets[j]['labels']
        predictions = lwpc.predict(features)
        accuracy = np.mean(predictions == labels)
        print(f"Accuracy on T{j+1}: {accuracy}")


Updating model for Dataset D11...
Accuracy on T1: 0.7464
Accuracy on T2: 0.7476
Accuracy on T3: 0.7292
Accuracy on T4: 0.7448
Accuracy on T5: 0.76
Accuracy on T6: 0.7588
Accuracy on T7: 0.7336
Accuracy on T8: 0.7412
Accuracy on T9: 0.7328
Accuracy on T10: 0.7492
Accuracy on T11: 0.5732
Updating model for Dataset D12...
Accuracy on T1: 0.7464
Accuracy on T2: 0.7476
Accuracy on T3: 0.7292
Accuracy on T4: 0.7448
Accuracy on T5: 0.76
Accuracy on T6: 0.7588
Accuracy on T7: 0.7336
Accuracy on T8: 0.7412
Accuracy on T9: 0.7328
Accuracy on T10: 0.7492
Accuracy on T11: 0.5732
Accuracy on T12: 0.4976
Updating model for Dataset D13...
Accuracy on T1: 0.7464
Accuracy on T2: 0.7476
Accuracy on T3: 0.7292
Accuracy on T4: 0.7448
Accuracy on T5: 0.76
Accuracy on T6: 0.7588
Accuracy on T7: 0.7336
Accuracy on T8: 0.7412
Accuracy on T9: 0.7328
Accuracy on T10: 0.7492
Accuracy on T11: 0.5732
Accuracy on T12: 0.4976
Accuracy on T13: 0.6756
Updating model for Dataset D14...
Accuracy on T1: 0.7464
Accuracy o